In [1]:
from pyvis.network import Network
import pandas as pd

In [2]:
fraudulent_device = '91b12379-8098-457f-a2ad-a94d767797c2'
fraudulent_account = '0007f265568f1abc1da791e852877df2047b3af9'
data = pd.read_csv('output.csv')

In [3]:
def prepare_nodes_parameters(df:pd.DataFrame, color: str = 'white') -> dict:
    result = {
        'n_id': [],
        'value': [],
        'title': [],
        'label': [],
        'color': [],
        'data': []
    }
    for index, row in df.iterrows():
        result['n_id'].append(index)
        result['value'].append(100)
        result['title'].append(
            f"""device_id: {row['device_id']}
            account: {row['identity']}
            device_fingerprint: {row['device_fingerprint']}
            bank: {row['bank']}
            ip: {row['ips']}
            lat: {row['lat']}
            lon: {row['lon']}
            country: {row['country']}
            city: {row['city']}
            os: {row['os']}
            browser: {row['browser']}
            screen: {row['screen']}
            mobile: {row['mobile']}
            proxy: {row['proxy']}
            """)
        result['label'].append(f'id {str(index)}')
        result['color'].append(color)
        result['data'].append(row[:].to_dict())
    return result

In [4]:
def calc_edge_width_and_title(row1: dict, row2: dict, fields: list = list(data.columns)) -> (int, str):
    """
    Compares each column of both nodes data and calculates how many connections they have.
    If there are connections in device_id, device_fingerprint, identitywill width grow by 10
    :return: integer value of edge width and title for edge.
    Title contains names of connection fields.
    """
    title = ['These fields are equal:']
    width = 0
    for field in fields:
        if row1[field] == row2[field]:
            title.append(field)
            width += 1
            if field in ['device_id', 'identity', 'device_fingerprint']:
                width += 9
    return width, '\n'.join(title)

In [5]:
def one_node_to_many_list(node1: dict, nodes_dict: dict) -> list:
    result = []
    for i in range(len(nodes_dict['n_id'])):
        width, title = calc_edge_width_and_title(node1[list(node1.keys())[0]], nodes_dict['data'][i])
        if width != 0:
            result.append((list(node1.keys())[0], nodes_dict['n_id'][i], width, title))
    return result

In [6]:
compromised_nodes = prepare_nodes_parameters(
    data[
        (data['device_id'] == fraudulent_device)
        | (data['identity'] == fraudulent_account)
    ],
    'red'
)

other_nodes = prepare_nodes_parameters(
    data[
        (data['device_id'] != fraudulent_device)
        & (data['identity'] != fraudulent_account)
    ],
    'blue'
)

In [20]:
# compromised_nodes

{'n_id': [595,
  596,
  628,
  630,
  633,
  637,
  639,
  642,
  644,
  652,
  653,
  1048,
  1050,
  1051,
  1054,
  1055,
  1056,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1066,
  1067,
  1068,
  1176,
  1177,
  1178,
  1179,
  1180,
  1181,
  1182,
  1183,
  1184,
  1185,
  1187,
  1189,
  1192,
  1193,
  1194,
  1196,
  1198,
  1199,
  1200,
  1201,
  1203,
  1205,
  1206,
  1208,
  1694,
  1695,
  1701,
  1702,
  1725,
  1726,
  1727,
  1728,
  1729,
  1730,
  1731,
  1758,
  1759,
  1760,
  1761,
  1762,
  1763,
  1764,
  1765,
  1766,
  1767,
  1768,
  1771,
  1772,
  1773,
  1774,
  1775],
 'value': [100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  10

In [7]:
edge_list = []

for i in range(len(compromised_nodes['n_id'])):
    edge_list += one_node_to_many_list({compromised_nodes['n_id'][i]: compromised_nodes['data'][i]}, other_nodes)
# edge_list

In [6]:
associated_fingerprints = list(set(data['device_fingerprint'][
    ((data['device_id'] == fraudulent_device)
    | (data['identity'] == fraudulent_account))
    & (data['device_fingerprint'] != 'No data')
].values))

In [7]:
parameters_of_screen = {
    'os': [],
    'browser': [],
    'screen': [],
}

for index, row in data[data['device_fingerprint'].isin(associated_fingerprints)].iterrows():
    parameters_of_screen['os'].append(row['os'])
    parameters_of_screen['browser'].append(row['browser'])
    parameters_of_screen['screen'].append(row['screen'])

for key in parameters_of_screen.keys():
    parameters_of_screen[key] = list(set(parameters_of_screen[key]))

similar_devices = 1

In [8]:
net = Network(height="750px", width="100%", bgcolor="#222230", font_color="white", notebook=True, cdn_resources='in_line')

In [9]:
net.add_nodes(
    compromised_nodes['n_id'], 
    value=compromised_nodes['value'],
    title=compromised_nodes['title'],
    label=compromised_nodes['label'],
    color=compromised_nodes['color']
)

In [10]:
net.add_nodes(
    other_nodes['n_id'], 
    value=other_nodes['value'],
    title=other_nodes['title'],
    label=other_nodes['label'],
    color=other_nodes['color']
)

In [11]:
# net.add_edge(1,2, title='hi', weight=.08)
for edge in edge_list:
    net.add_edge(edge[0], edge[1], width=edge[2], title=edge[3])

In [12]:
net.edges

[{'width': 4,
  'title': 'These fields are equal:\ncountry\ncountryCode\nproxy\nhosting',
  'from': 595,
  'to': 0},
 {'width': 3,
  'title': 'These fields are equal:\nmobile\nproxy\nhosting',
  'from': 595,
  'to': 1},
 {'width': 5,
  'title': 'These fields are equal:\ncountry\ncountryCode\nmobile\nproxy\nhosting',
  'from': 595,
  'to': 2},
 {'width': 4,
  'title': 'These fields are equal:\ncountry\ncountryCode\nproxy\nhosting',
  'from': 595,
  'to': 3},
 {'width': 4,
  'title': 'These fields are equal:\ncountry\ncountryCode\nproxy\nhosting',
  'from': 595,
  'to': 4},
 {'width': 5,
  'title': 'These fields are equal:\ncountry\ncountryCode\nmobile\nproxy\nhosting',
  'from': 595,
  'to': 5},
 {'width': 4,
  'title': 'These fields are equal:\ncountry\ncountryCode\nproxy\nhosting',
  'from': 595,
  'to': 6},
 {'width': 5,
  'title': 'These fields are equal:\ncountry\ncountryCode\nmobile\nproxy\nhosting',
  'from': 595,
  'to': 7},
 {'width': 4,
  'title': 'These fields are equal:\ncou

In [13]:
net.show('nx.html')

nx.html
